In [1]:
# Multi-task Deep Learning for Oral Cancer Analysis
# ==================================================
# This notebook sets up the project environment and demonstrates data loading

print("🔬 Multi-task Deep Learning for Oral Cancer Analysis")
print("=" * 60)

# Import system packages
import sys
import os
from pathlib import Path

# Add src directory to path for imports
project_root = Path.cwd().parent if Path.cwd().name == "Source Code" else Path.cwd()
src_path = project_root / "src"
sys.path.append(str(src_path))

print(f"Project root: {project_root}")
print(f"Current working directory: {Path.cwd()}")

# Test configuration import
try:
    from config import get_config, print_config_summary
    print("✅ Configuration module loaded successfully")
    
    # Print configuration summary
    print_config_summary()
    
except ImportError as e:
    print(f"❌ Could not import configuration: {e}")
    print("Please ensure you've run the setup script first")

🔬 Multi-task Deep Learning for Oral Cancer Analysis
Project root: c:\Users\user\Desktop\SEGP\Multi-task-Deep-Learning-for-Quantifying-Key-Histopathological-Features-in-Oral-Cancer
Current working directory: c:\Users\user\Desktop\SEGP\Multi-task-Deep-Learning-for-Quantifying-Key-Histopathological-Features-in-Oral-Cancer\Source Code
✅ Configuration module loaded successfully


AssertionError: Torch not compiled with CUDA enabled

# Environment Setup and Verification

Before starting the analysis, let's verify that our environment is properly configured for multi-task deep learning on oral cancer histopathology data.

## Key Components:
- **TVNT**: Tumour vs Non-Tumour classification
- **DOI**: Depth of Invasion measurement  
- **POI**: Pattern of Invasion classification
- **TB**: Tumour Budding detection
- **PNI**: Perineural Invasion detection
- **MI**: Mitotic Index detection

## Hardware Requirements:
- Local: RTX 4050 (6GB VRAM) - for prototyping
- Cloud: Colab/Kaggle - for full training

In [2]:
# Check PyTorch and GPU availability
import torch
import torchvision
print(f"PyTorch version: {torch.__version__}")
print(f"TorchVision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Memory optimization for RTX 4050
    if "4050" in torch.cuda.get_device_name(0):
        print("\n💡 RTX 4050 Optimization Tips:")
        print("- Use batch_size <= 16 for training")
        print("- Enable mixed precision (AMP)")
        print("- Use gradient checkpointing for large models")
        print("- Consider patch size <= 512px")
else:
    print("⚠️  Running on CPU - consider using GPU for better performance")

PyTorch version: 2.8.0+cpu
TorchVision version: 0.23.0+cpu
CUDA available: False
⚠️  Running on CPU - consider using GPU for better performance


In [5]:
# Import essential libraries for medical image analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

print("✅ Core libraries imported successfully")

# Check for medical image processing libraries
try:
    import openslide
    print("✅ OpenSlide available for WSI processing")
except ImportError:
    print("❌ OpenSlide not found - needed for WSI processing")
    print("Install with: pip install openslide-python")

try:
    import albumentations as A
    print("✅ Albumentations available for data augmentation")
except ImportError:
    print("❌ Albumentations not found - needed for augmentation")
    print("Install with: pip install albumentations")

# Display versions for reproducibility
print(f"\nLibrary Versions (for reproducibility):")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")

✅ Core libraries imported successfully
❌ OpenSlide not found - needed for WSI processing
Install with: pip install openslide-python
✅ Albumentations available for data augmentation

Library Versions (for reproducibility):
NumPy: 2.2.6
Pandas: 2.3.2
OpenCV: 4.12.0
Matplotlib: 3.10.6
Seaborn: 0.13.2


# Data Exploration

Let's explore the existing data structure and understand what we're working with.

In [6]:
# Explore project structure
project_root = Path.cwd().parent
print("📁 Project Structure:")
print("=" * 40)

def print_tree(directory, prefix="", max_depth=3, current_depth=0):
    """Print directory tree structure"""
    if current_depth > max_depth:
        return
    
    items = sorted(directory.iterdir()) if directory.is_dir() else []
    folders = [item for item in items if item.is_dir()]
    files = [item for item in items if item.is_file()]
    
    # Print folders first
    for i, folder in enumerate(folders):
        is_last_folder = (i == len(folders) - 1) and len(files) == 0
        current_prefix = "└── " if is_last_folder else "├── "
        print(f"{prefix}{current_prefix}{folder.name}/")
        
        # Recursive call for subdirectories
        extension = "    " if is_last_folder else "│   "
        print_tree(folder, prefix + extension, max_depth, current_depth + 1)
    
    # Then print files (limited)
    for i, file in enumerate(files[:5]):  # Show first 5 files
        is_last = i == len(files) - 1 or i == 4
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{file.name}")
    
    if len(files) > 5:
        print(f"{prefix}└── ... ({len(files) - 5} more files)")

print_tree(project_root)

📁 Project Structure:
├── .git/
│   ├── hooks/
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   └── pre-applypatch.sample
│   │   └── ... (9 more files)
│   ├── info/
│   │   └── exclude
│   ├── logs/
│   │   ├── refs/
│   │   │   ├── heads/
│   │   │   └── remotes/
│   │   └── HEAD
│   ├── objects/
│   │   ├── 00/
│   │   │   ├── 0a96d2498d813494b7c55bb7fdf536b129c185
│   │   │   └── 2fa5d5893470e41aaaadbdbc8187433801b652
│   │   ├── 01/
│   │   │   ├── 059ed90fb549bedba8d405f4fd5e88def690d0
│   │   │   ├── 233097058e48ae377b17066de1f5ab68c55b62
│   │   │   ├── 294e747675f05a59f42bed607960d6609d66dd
│   │   │   ├── 59db33652afe2cb428c67158ac3ff166256cc6
│   │   │   └── 92781a49f30ebe5cf658e12517b7e6e4da0562
│   │   │   └── ... (1 more files)
│   │   ├── 02/
│   │   │   ├── 1b6ffe5a40dcf6e38c9ab7833e8e4dda036702
│   │   │   ├── 5a11c849ff8361762ce41b2a668def24f485e9
│   │   │   ├── 6c359655b7e387

# DenseNet-169 + K-Net Multi-task Model

Now let's demonstrate the combined DenseNet-169 classification backbone with K-Net segmentation model for multi-task learning in oral cancer analysis.

## Model Architecture Overview:
- **Backbone**: DenseNet-169 (pretrained on ImageNet)
- **Classification Head**: Multi-task heads for TVNT, POI, PNI, DOI
- **Segmentation Head**: K-Net with dynamic kernels for tissue segmentation
- **Feature Sharing**: Shared backbone with task-specific adaptations

In [7]:
# Import the model architectures
try:
    from src.models import (
        create_densenet169_model,
        create_knet_model, 
        create_multitask_model,
        get_available_models
    )
    print("✅ Model architectures imported successfully")
    
    # Show available models
    available_models = get_available_models()
    print(f"\nAvailable Models:")
    for category, models in available_models.items():
        print(f"  {category.title()}: {', '.join(models)}")
        
except ImportError as e:
    print(f"❌ Could not import models: {e}")
    print("Make sure you've set up the models directory correctly")

❌ Could not import models: No module named 'src'
Make sure you've set up the models directory correctly


In [8]:
# Test the updated configuration with new models
try:
    from src.config import get_config, print_config_summary
    
    config = get_config()
    print("🔧 Updated Configuration for DenseNet-169 + K-Net:")
    print("=" * 60)
    
    model_config = config['model']
    
    print(f"Classification Tasks: {list(model_config.classification_tasks.keys())}")
    print(f"Segmentation Tasks: {len(model_config.segmentation_tasks)} classes")
    print(f"Default Backbone: {model_config.default_backbone}")
    print(f"K-Net Kernels: {model_config.knet_num_kernels}")
    print(f"Feature Sharing: {model_config.feature_sharing}")
    print(f"Task Attention: {model_config.use_task_attention}")
    
    print(f"\nTask Weights:")
    for task, weight in model_config.task_weights.items():
        print(f"  {task}: {weight}")
        
except ImportError as e:
    print(f"❌ Configuration import error: {e}")

❌ Configuration import error: No module named 'src'


In [9]:
# Create and test the combined DenseNet-169 + K-Net model
print("🏗️  Creating Combined Multi-task Model (DenseNet-169 + K-Net)")
print("=" * 70)

try:
    # Create the combined model
    model = create_multitask_model(pretrained=True)
    
    # Model statistics
    model_stats = model.compute_model_size()
    print("Model Architecture Statistics:")
    print(f"  • Total Parameters: {model_stats['total_parameters']:,}")
    print(f"  • Backbone Parameters: {model_stats['backbone_parameters']:,}")
    print(f"  • Classification Parameters: {model_stats['classification_parameters']:,}")
    print(f"  • Segmentation Parameters: {model_stats['segmentation_parameters']:,}")
    
    # Calculate model size in MB
    total_params = model_stats['total_parameters']
    model_size_mb = (total_params * 4) / (1024 * 1024)  # 4 bytes per parameter
    print(f"  • Estimated Model Size: {model_size_mb:.1f} MB")
    
    # Check if model fits in local GPU memory
    if torch.cuda.is_available():
        gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"  • Available GPU Memory: {gpu_memory_gb:.1f} GB")
        
        if model_size_mb < (gpu_memory_gb * 1000 * 0.5):  # Use 50% of GPU memory as threshold
            print("  ✅ Model should fit in local GPU memory")
        else:
            print("  ⚠️  Model may require cloud training or memory optimization")
    
    print(f"\nModel created successfully! 🎉")
    
except Exception as e:
    print(f"❌ Error creating model: {e}")
    print("This might be due to missing dependencies or configuration issues")

🏗️  Creating Combined Multi-task Model (DenseNet-169 + K-Net)
❌ Error creating model: name 'create_multitask_model' is not defined
This might be due to missing dependencies or configuration issues


In [10]:
# Test model with dummy input data
print("🧪 Testing Model with Dummy Data")
print("=" * 40)

if 'model' in locals():
    try:
        # Create dummy input (batch_size=2, channels=3, height=512, width=512)
        dummy_input = torch.randn(2, 3, 512, 512)
        print(f"Input shape: {dummy_input.shape}")
        
        # Move to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        dummy_input = dummy_input.to(device)
        
        print(f"Using device: {device}")
        
        # Forward pass
        with torch.no_grad():
            outputs = model(dummy_input, return_features=True)
        
        print("\n📊 Model Output Analysis:")
        
        # Classification outputs
        print("\n🎯 Classification Outputs:")
        for task_name, output in outputs['classification'].items():
            print(f"  {task_name}: {output.shape}")
            if model.classification_head.tasks_config[task_name]['type'] == 'binary_classification':
                probs = torch.softmax(output, dim=1)
                print(f"    Sample probabilities: {probs[0].cpu().numpy()}")
            elif model.classification_head.tasks_config[task_name]['type'] == 'regression':
                print(f"    Sample prediction: {output[0].item():.3f}")
        
        # Segmentation outputs
        print("\n🗺️  Segmentation Outputs:")
        seg_logits = outputs['segmentation']['seg_logits']
        print(f"  Segmentation logits: {seg_logits.shape}")
        
        # Convert to probabilities
        seg_probs = torch.softmax(seg_logits, dim=1)
        print(f"  Segmentation probabilities: {seg_probs.shape}")
        
        # Get predicted classes
        seg_pred = torch.argmax(seg_probs, dim=1)
        print(f"  Predicted classes: {seg_pred.shape}")
        print(f"  Unique classes in prediction: {torch.unique(seg_pred).cpu().numpy()}")
        
        print("\n✅ Model forward pass completed successfully!")
        
        # Memory usage
        if torch.cuda.is_available():
            memory_used = torch.cuda.max_memory_allocated() / 1e9
            print(f"  GPU Memory Used: {memory_used:.2f} GB")
        
    except Exception as e:
        print(f"❌ Error during forward pass: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Model not available. Please run the model creation cell first.")

🧪 Testing Model with Dummy Data
❌ Model not available. Please run the model creation cell first.


In [11]:
# Demonstrate individual model components
print("🔍 Individual Model Components Demo")
print("=" * 50)

try:
    # 1. DenseNet-169 Classification only
    print("\n1️⃣ DenseNet-169 Classification Model:")
    cls_model = create_densenet169_model(pretrained=True)
    
    # Test classification model
    with torch.no_grad():
        dummy_input_cls = torch.randn(1, 3, 512, 512)
        if torch.cuda.is_available():
            cls_model = cls_model.cuda()
            dummy_input_cls = dummy_input_cls.cuda()
        
        cls_outputs = cls_model(dummy_input_cls)
        
        print(f"  Input shape: {dummy_input_cls.shape}")
        print("  Classification outputs:")
        for task, output in cls_outputs.items():
            print(f"    {task}: {output.shape}")
    
    # 2. K-Net Segmentation only  
    print("\n2️⃣ K-Net Segmentation Model:")
    seg_model = create_knet_model(num_classes=6)
    
    # Test segmentation model with backbone features
    with torch.no_grad():
        # Simulate backbone features (DenseNet-169 output)
        backbone_features = torch.randn(1, 1664, 16, 16)  # DenseNet-169 feature size
        if torch.cuda.is_available():
            seg_model = seg_model.cuda()
            backbone_features = backbone_features.cuda()
        
        seg_outputs = seg_model(backbone_features)
        
        print(f"  Backbone features shape: {backbone_features.shape}")
        print("  Segmentation outputs:")
        for key, value in seg_outputs.items():
            if isinstance(value, torch.Tensor):
                print(f"    {key}: {value.shape}")
            elif isinstance(value, list):
                print(f"    {key}: {len(value)} stages")
    
    # 3. Model comparison
    print("\n📊 Model Comparison:")
    cls_params = sum(p.numel() for p in cls_model.parameters())
    seg_params = sum(p.numel() for p in seg_model.parameters()) 
    combined_params = sum(p.numel() for p in model.parameters())
    
    print(f"  DenseNet-169 Classification: {cls_params:,} parameters")
    print(f"  K-Net Segmentation: {seg_params:,} parameters")
    print(f"  Combined Multi-task: {combined_params:,} parameters")
    
    # Feature sharing advantage
    separate_total = cls_params + seg_params
    sharing_efficiency = (1 - combined_params / separate_total) * 100
    print(f"  Parameter efficiency from sharing: {sharing_efficiency:.1f}% reduction")
    
except Exception as e:
    print(f"❌ Error in component demo: {e}")
    import traceback
    traceback.print_exc()

🔍 Individual Model Components Demo

1️⃣ DenseNet-169 Classification Model:
❌ Error in component demo: name 'create_densenet169_model' is not defined


Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_11776\2352086100.py", line 8, in <module>
    cls_model = create_densenet169_model(pretrained=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'create_densenet169_model' is not defined


In [19]:
# Add this as a new test cell
print("🔧 Testing Fixed Multitask Model with Feature Adaptation")
print("=" * 55)

import sys
from pathlib import Path
import torch

# Setup paths and clear cache
project_root = Path.cwd().parent if Path.cwd().name == "Source Code" else Path.cwd()
src_path = project_root / "src"
models_path = src_path / "models"

# Clear cached imports
for module in ['multitask_model']:
    if module in sys.modules:
        del sys.modules[module]

sys.path.insert(0, str(src_path))
sys.path.insert(0, str(models_path))

try:
    # Import the fixed multitask model
    import multitask_model
    print("✅ Multitask model imported")
    
    # Create the model
    print("\n🏗️ Creating multitask model...")
    model = multitask_model.create_multitask_model(pretrained=True)
    print("✅ Model created successfully!")
    
    # Test forward pass with debug info
    print("\n🧪 Testing forward pass with feature adaptation...")
    dummy_input = torch.randn(1, 3, 512, 512)
    
    with torch.no_grad():
        outputs = model(dummy_input, return_features=True)
    
    print("✅ Forward pass successful!")
    print(f"   Classification tasks: {list(outputs['classification'].keys())}")
    
    # Check segmentation output
    if 'seg_logits' in outputs['segmentation']:
        seg_shape = outputs['segmentation']['seg_logits'].shape
        print(f"   Segmentation logits: {seg_shape}")
    else:
        print(f"   Segmentation keys: {list(outputs['segmentation'].keys())}")
    
    # Model statistics
    stats = model.compute_model_size()
    print(f"\n📊 Model Statistics:")
    print(f"   Total parameters: {stats['total_parameters']:,}")
    print(f"   Backbone: {stats['backbone_parameters']:,}")
    print(f"   Classification: {stats['classification_parameters']:,}")
    print(f"   Segmentation: {stats['segmentation_parameters']:,}")
    print(f"   Feature adapter: {stats['adapter_parameters']:,}")
    
    # Calculate model size
    model_size_mb = (stats['total_parameters'] * 4) / (1024 * 1024)
    print(f"   Model size: {model_size_mb:.1f} MB")
    
    print("\n🎉 All tests passed successfully!")
    print("🚀 Multi-task DenseNet-169 + K-Net model with feature adaptation is ready!")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    print("\nDetailed traceback:")
    traceback.print_exc()

🔧 Testing Fixed Multitask Model with Feature Adaptation
✅ Multitask model imported

🏗️ Creating multitask model...
Creating DenseNet backbone...
   Attempt 1 failed: DenseNetMultiTask.__init__() got an unexpected keyword argument 'use_attention'
✅ Created with pretrained=True
✅ Found classification_heads attribute
Creating K-Net segmentation head...
   K-Net attempt 1 failed: KNetSegmentation.__init__() got an unexpected keyword argument 'backbone_dim'
   K-Net attempt 2 failed: KNetSegmentation.__init__() got an unexpected keyword argument 'input_dim'
✅ K-Net created with parameters: {'num_classes': 6, 'num_kernels': 100}
✅ Segmentation feature adapter created (1664→256 channels, 2x upsampling)
✅ Model created successfully!

🧪 Testing forward pass with feature adaptation...
✅ Created with pretrained=True
✅ Found classification_heads attribute
Creating K-Net segmentation head...
   K-Net attempt 1 failed: KNetSegmentation.__init__() got an unexpected keyword argument 'backbone_dim'
   